# Forecasting de Ventas 2025
Este notebook importa las librerías principales y carga los datos de ventas para inferencia.

In [2]:
# Imports principales para el forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

## Cargar archivo de ventas para inferencia
Se carga el archivo ventas_2025_inferencia.csv en un DataFrame llamado inferencia_df.

In [5]:
# Cargar archivo de ventas para inferencia
inferencia_path = '../data/raw/inferencia/ventas_2025_inferencia.csv'
inferencia_df = pd.read_csv(inferencia_path)
print('Primeras filas del DataFrame de inferencia:')
display(inferencia_df.head())

Primeras filas del DataFrame de inferencia:


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [6]:
# --- TRANSFORMACIÓN DE inferencia_df PARA INFERENCIA ---

# 1. Conversión de tipos
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Variables temporales y de calendario
inferencia_df['anio'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['mes_nombre'] = inferencia_df['fecha'].dt.month_name()
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin(['Saturday', 'Sunday'])

# Festivos en España
es_festivo = inferencia_df['fecha'].isin([d for d in holidays.country_holidays('ES', years=inferencia_df['anio'].unique())])
inferencia_df['es_festivo'] = es_festivo.values

# Black Friday (último viernes de noviembre)
def es_black_friday(fecha):
    if fecha.month == 11 and fecha.weekday() == 4:
        return (fecha + pd.Timedelta(days=7)).month == 12
    return False
inferencia_df['es_Black_Friday'] = inferencia_df['fecha'].apply(es_black_friday)

# Cyber Monday (primer lunes después de Black Friday)
def es_cyber_monday(fecha):
    if fecha.month == 11 and fecha.weekday() == 0:
        ultimo_viernes = max([d for d in inferencia_df['fecha'] if d.month == 11 and d.weekday() == 4 and d.year == fecha.year])
        return fecha == (ultimo_viernes + pd.Timedelta(days=3))
    elif fecha.month == 12 and fecha.weekday() == 0:
        ultimo_viernes = max([d for d in inferencia_df['fecha'] if d.month == 11 and d.weekday() == 4 and d.year == fecha.year])
        return fecha == (ultimo_viernes + pd.Timedelta(days=3))
    return False
inferencia_df['es_cyber_Monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

inferencia_df['dia_anio'] = inferencia_df['fecha'].dt.dayofyear
inferencia_df['semana_anio'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['es_principio_mes'] = inferencia_df['dia_mes'] <= 3
inferencia_df['es_fin_mes'] = inferencia_df['fecha'] == inferencia_df['fecha'] + pd.offsets.MonthEnd(0)

# 3. Variables de competencia y ratio de precios
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
inferencia_df = inferencia_df.drop(columns=competidores)

# 4. Variable descuento porcentaje
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# 5. One hot encoding de variables categóricas
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']

one_hot = pd.get_dummies(inferencia_df[['nombre_h', 'categoria_h', 'subcategoria_h']], prefix=['nombre_h', 'categoria_h', 'subcategoria_h'])
inferencia_df = pd.concat([inferencia_df, one_hot], axis=1)

# 6. Eliminar registros de octubre y dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['fecha'].dt.month == 11].copy()

# 7. Guardar el DataFrame transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print("inferencia_df transformado y guardado en ../data/processed/inferencia_df_transformado.csv")

inferencia_df transformado y guardado en ../data/processed/inferencia_df_transformado.csv


C:\Users\lucia\AppData\Local\Temp\ipykernel_5464\3851890200.py:15: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  es_festivo = inferencia_df['fecha'].isin([d for d in holidays.country_holidays('ES', years=inferencia_df['anio'].unique())])


## Resumen

In [10]:
# Guardar el DataFrame transformado en data/processed
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print("DataFrame transformado guardado en data/processed/inferencia_df_transformado.csv")

# Mostrar resumen final
print(f"\nResumen final:")
print(f"Total de registros: {len(inferencia_df)}")
print(f"Total de columnas: {len(inferencia_df.columns)}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")
print(f"Rango de fechas: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")


DataFrame transformado guardado en data/processed/inferencia_df_transformado.csv

Resumen final:
Total de registros: 720
Total de columnas: 74
Productos únicos: 24
Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00


In [11]:
inferencia_df.columns

Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'anio', 'mes', 'mes_nombre', 'dia_mes', 'dia_semana',
       'es_fin_semana', 'es_festivo', 'es_Black_Friday', 'es_cyber_Monday',
       'dia_anio', 'semana_anio', 'trimestre', 'es_principio_mes',
       'es_fin_mes', 'precio_competencia', 'ratio_precio',
       'descuento_porcentaje', 'nombre_h', 'categoria_h', 'subcategoria_h',
       'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23',
       'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552',
       'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas Set', 'nombre_h_Domyos BM900',
       'nombre_h_Domyos Kit Mancuernas 20kg',
       'nombre_h_Gaiam Premium Yoga Block', 'nombre_h_Liforme Yoga Pad',
       'nombre_h_Lotuscrafts Yoga Bolster', 'nombre_h_Manduka PRO Yoga Mat',
       'nombre_h_Merrell Moab 2 GTX',
   